In [2]:
import torch

from fastai.vision.all import *

import pandas as pd
import numpy as np
from sklearn.metrics import precision_recall_curve, accuracy_score
import matplotlib.pyplot as plt

In [3]:
print(torch.cuda.is_available())

True


### Helper Functions


In [4]:
# Plotting Precision-Recall curve for a specific class
def plot_precision_recall_curve(class_index, preds, targets):
    precision, recall, _ = precision_recall_curve(
        targets.numpy()[:, class_index], preds.numpy()[:, class_index]
    )
    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, marker=".")
    plt.title(f"Precision-Recall Curve for Class {class_index}")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.show()


def per_class_accuracy(
    class_index,
    preds,
    targets,
):
    class_preds = preds.argmax(dim=1)
    class_targets = targets == class_index
    class_preds = class_preds[class_targets]
    acc = accuracy_score(class_targets.numpy(), class_preds.numpy())
    return acc

In [ ]:
class_names = np.load("class_names.npy", allow_pickle=True)
# print(class_names.item())

attributes = np.load("attributes.npy", allow_pickle=True)
attributes.shape

In [ ]:
train_df = pd.read_csv("train_images.csv")
test_df = pd.read_csv("test_images_path.csv")

dls = ImageDataLoaders.from_df(
    train_df,
    valid_pct=0.2,
    item_tfms=Resize(224),
    bs=64,
    label_col="label",
    fn_col="image_path",
)

In [ ]:
dls.train.item_tfms = [
    RandomResizedCrop(224, min_scale=0.5),
    Rotate(),
    Flip(),
    Dihedral(),
    Brightness(),
    Contrast(),
    Saturation(),
    Hue(),
]

## Modelling


In [ ]:
learn = vision_learner(dls, resnet34, metrics=accuracy)

In [ ]:
learn.lr_find()

In [ ]:
learn.fine_tune(4)

#### Visualization


In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(20, 20), dpi=60)

In [ ]:
interp.plot_top_losses(9, figsize=(15, 10))

In [ ]:
learn.recorder.plot_metrics()

In [ ]:
learn.show_results()

## Testing


In [ ]:
test_dl = dls.test_dl(test_df, with_labels=True)
predictions, targets = learn.get_preds(dl=test_dl)

In [ ]:
acc = accuracy(predictions, targets)
print(f"Accuracy: {acc.item() * 100:.2f}%")

In [ ]:
# Example for class 0
plot_precision_recall_curve(0, predictions, targets)

In [ ]:
# Example accuracy for class 0
acc_class_0 = per_class_accuracy(predictions, targets, 0)
print(f"Accuracy for class 0: {acc_class_0:.2f}")

In [ ]:
class_preds = np.argmax(predictions, axis=1)
plt.hist(class_preds, bins=len(dls.vocab))
plt.title("Histogram of Predicted Classes")
plt.xlabel("Class")
plt.ylabel("Number of Predictions")
plt.show()

## Results


## Creating Sample


In [ ]:
# Create a submission DataFrame
submission_df = test_df.copy()
submission_df["label"] = predicted_labels

# Optional: if your test_df contains a column that represents an ID, use that instead
# For example, if there's an 'image_id' column:
# submission_df = pd.DataFrame({"id": test_df["image_id"], "label": predicted_labels})

# Save the submission file
submission_df[["id", "label"]].to_csv("submission.csv", index=False)

In [ ]:
torch.cuda.empty_cache()